In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/kface/Middle_Resolution.zip'
model_path = '/content/drive/MyDrive/Colab Notebooks/deep_learning/model/yolov5_kface'

# YOLOv5 모델 다운로드

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 14887, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 14887 (delta 0), reused 0 (delta 0), pack-reused 14882
Receiving objects: 100% (14887/14887), 13.86 MiB | 11.51 MiB/s, done.
Resolving deltas: 100% (10246/10246), done.
/content/yolov5
     |████████████████████████████████| 182 kB 6.2 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 1.6 MB 41.0 MB/s 


# 데이터셋 다운로드

In [ ]:
import zipfile
%mkdir /content/yolov5/kface
%cd /content/yolov5/kface
path = data_path
data = zipfile.ZipFile(path, 'r')
data.extractall('/content/yolov5/kface')
data.close()
# !unzip '/content/drive/MyDrive/Colab Notebooks/kface/Middle_Resolution.zip'

/content/yolov5/kface


# 바운딩 박스 좌표 정규화

In [ ]:
import os
import glob
import shutil
from tqdm import tqdm

In [ ]:
img_size = (230, 346) # (h, w)
classes=['S001_E01', 'S001_E02', 'S001_E03', 'S002_E01', 'S002_E02', 'S002_E03',
         'S003_E01', 'S003_E02', 'S003_E03', 'S004_E01', 'S004_E02', 'S004_E03',
         'S005_E01', 'S005_E02', 'S005_E03', 'S006_E01', 'S006_E02', 'S006_E03']

In [ ]:
txt_paths = sorted(glob.glob('/content/yolov5/kface/*/*.txt'))
jpg_paths = sorted(glob.glob('/content/yolov5/kface/*/*.jpg'))

In [ ]:
for txt_path in tqdm(txt_paths):
    filename = txt_path.split('/')[-1]
    label = txt_path.split('/')[-2]
    num_label = classes.index(label)

    with open(txt_path, 'rt') as f:
        lines = f.readlines()

    x, y, w, h = map(int, lines[7].split('\t'))
    x = (x+w/2)/img_size[1]
    y = (y+h/2)/img_size[0]
    w = w/img_size[1]
    h = h/img_size[0]

    target_path = '/content/yolov5/kface/train/labels/'+label+'_'+filename
    if not os.path.isdir('/content/yolov5/kface/train/labels/'):
        os.makedirs('/content/yolov5/kface/train/labels/')
    with open(target_path, 'wt') as f:
        f.write(f'{num_label} {x} {y} {w} {h}')

100%|██████████| 93600/93600 [00:20<00:00, 4526.53it/s]


In [ ]:
for jpg_path in tqdm(jpg_paths):
    filename = jpg_path.split('/')[-1]
    label = jpg_path.split('/')[-2]

    target_path = '/content/yolov5/kface/train/images/'+label+'_'+filename
    if not os.path.isdir('/content/yolov5/kface/train/images/'):
        os.makedirs('/content/yolov5/kface/train/images/')
    shutil.move(jpg_path, target_path)

100%|██████████| 93600/93600 [00:04<00:00, 21025.84it/s]


In [ ]:
# 폴더 정리
for cls in classes:
    shutil.rmtree('/content/yolov5/kface/'+cls+'/')

# 데이터 나누기

In [ ]:
def split_img_label(data_valid, folder_valid):
    
    os.makedirs(folder_valid+'images/')
    os.makedirs(folder_valid+'labels/')

    valid_ind = list(data_valid.index)

    # Valid folder
    for j in tqdm(range(len(valid_ind))):
        shutil.move(data_valid[valid_ind[j]], folder_valid+'images/'+data_valid[valid_ind[j]].split('/')[-1])
        shutil.move('/'+os.path.join(*data_valid[valid_ind[j]].split('/')[:-2])+'/labels/'+data_valid[valid_ind[j]].split('/')[-1].split('.jpg')[0]+'.txt', folder_valid+'labels/'+data_valid[valid_ind[j]].split('/')[-1].split('.jpg')[0]+'.txt')

In [ ]:
import pandas as pd 
import os
from sklearn.model_selection import train_test_split

txt_paths = sorted(glob.glob('/content/yolov5/kface/train/labels/*.txt'))
jpg_paths = sorted(glob.glob('/content/yolov5/kface/train/images/*.jpg'))

labels = []
for txt_path in txt_paths:
    with open(txt_path, 'rt') as f:
        lines = f.readlines()
    labels.append(lines[0].split(' ')[0])

labels = pd.DataFrame(labels)
df = pd.DataFrame(jpg_paths)
df = pd.concat([df, labels], axis=1)
df.columns = [0, 1]

In [ ]:
# split 
data_train, data_valid, labels_train, labels_valid = train_test_split(df[0], df[1], test_size=0.2, stratify=df[1], random_state=42)

folder_valid_name = '/content/yolov5/kface/valid/'

# Function split 
split_img_label(data_valid, folder_valid_name)

100%|██████████| 18720/18720 [00:02<00:00, 8304.60it/s]


In [ ]:
# # 데이터 저장
# file_path = '/content/yolov5/kface/'
# data = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/kface/data.zip', 'w')
# for (path, dir, files) in tqdm(os.walk(file_path)):
#     for file in files:
#         data.write(os.path.join(path, file), compress_type=zipfile.ZIP_DEFLATED)
# data.close()

# yolo데이터 준비

In [ ]:
train_img_list = glob.glob('/content/yolov5/kface/train/images/*.jpg')
valid_img_list = glob.glob('/content/yolov5/kface/valid/images/*.jpg')
len(train_img_list), len(valid_img_list)

(74880, 18720)

In [ ]:
# 파일 목록 실제 파일로 저장
with open('/content/yolov5/kface/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list)+'\n')

with open('/content/yolov5/kface/valid.txt', 'w') as f:
    f.write('\n'.join(valid_img_list)+'\n')

In [ ]:
# yaml 파일 수정을 위한 함수
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/kface/data.yaml

train: ../kface/train/images
val: ../kface/valid/images

nc: 18
names: ['None_Netural', 'None_Happy', 'None_Frown', 'Nomal_Glass_Netural', 'Nomal_Glass_Happy', 'Nomal_Glass_Frown',
        'Horn_rimmed_Glass_Netural', 'Horn_rimmed_Glass_Happy', 'Horn_rimmed_Glass_Frown',
        'Sunglasses_Netural', 'Sunglasses_Happy', 'Sunglasses_Frown',
        'Netual_with_cap', 'Happy_with_cap', 'Frown_with_cap',
        'Horn_rimmed_Glass_Netural_with_cap', 'Horn_rimmed_Glass_Happy_with_cap', 'Horn_rimmed_Glass_Frown_with_cap']

In [ ]:
%cat /content/yolov5/kface/data.yaml


train: ../kface/train/images
val: ../kface/valid/images

nc: 18
names: ['None_Netural', 'None_Happy', 'None_Frown', 'Nomal_Glass_Netural', 'Nomal_Glass_Happy', 'Nomal_Glass_Frown',
        'Horn_rimmed_Glass_Netural', 'Horn_rimmed_Glass_Happy', 'Horn_rimmed_Glass_Frown',
        'Sunglasses_Netural', 'Sunglasses_Happy', 'Sunglasses_Frown',
        'Netual_with_cap', 'Happy_with_cap', 'Frown_with_cap',
        'Horn_rimmed_Glass_Netural_with_cap', 'Horn_rimmed_Glass_Happy_with_cap', 'Horn_rimmed_Glass_Frown_with_cap']


# yolo 모델 구성

In [ ]:
import yaml

with open('/content/yolov5/kface/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc']) # 클래스 갯수 불러오기
num_classes

'18'

In [ ]:
# 사본 커스텀 저장
%%writetemplate /content/yolov5/models/kface_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# 수정 확인
%cat /content/yolov5/models/kface_yolov5s.yaml


# Parameters
nc: 18  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, F

In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
%%time
!python train.py --img 346 --batch 32 --epochs 20 --data ./kface/data.yaml \
--cfg ./models/kface_yolov5s.yaml --name kface_result --cache

train: weights=yolov5s.pt, cfg=./models/kface_yolov5s.yaml, data=./kface/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=346, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=kface_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-49-g3c1afd9 Python-3.8.16 torch-1.13.0+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

In [ ]:
!ls /content/yolov5/runs/train/kface_result

events.out.tfevents.1672072191.be5713d6e51b.1117.0  train_batch0.jpg
hyp.yaml					    train_batch1.jpg
labels_correlogram.jpg				    train_batch2.jpg
labels.jpg					    weights
opt.yaml


In [ ]:
from IPython.display import Image
Image(filename='/content/yolov5/runs/train/kface_result/results.png', width=800)

FileNotFoundError: ignored

In [ ]:
Image(filename='/content/yolov5/runs/train/kface_result/train_batch0.jpg', width=800)

In [ ]:
Image(filename='/content/yolov5/runs/train/kface_result/val_batch0_labels.jpg', width=800)

# 검증

In [ ]:
# validation data
!python val.py --weights runs/train/kface_result/weights/best.pt \
--data ./kface/data.yaml --img 864 --iou 0.65 --half

In [ ]:
# # test data
# !python val.py --weights runs/train/kface_result/weights/best.pt \
# --data ./kface/data.yaml --img 864 --task test

# 모델 저장

In [ ]:
%cp /content/yolov5/runs/train/kface_result3/weights/best.pt model_path